<a href="https://colab.research.google.com/github/bedwardsr1-boop/Proyecto-RTF-Tumor/blob/main/Proyecto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prueba

In [1]:
!git push

fatal: not a git repository (or any of the parent directories): .git
